In [1]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="krankenhaus",
    user="postgres",
    password="postgres"
)

cur = conn.cursor()


In [11]:
drop_tables = """
-- =========================
-- DROP TABLES
-- =========================

DROP TABLE IF EXISTS Medikament_Event CASCADE;
DROP TABLE IF EXISTS Event_Arzt CASCADE;
DROP TABLE IF EXISTS Event CASCADE;
DROP TABLE IF EXISTS Raum CASCADE;
DROP TABLE IF EXISTS Behandlung CASCADE;
DROP TABLE IF EXISTS Bett CASCADE;
DROP TABLE IF EXISTS Reserve CASCADE;
DROP TABLE IF EXISTS Pflegekraft CASCADE;
DROP TABLE IF EXISTS Arzt CASCADE;
DROP TABLE IF EXISTS Patient CASCADE;
DROP TABLE IF EXISTS Station CASCADE;
DROP TABLE IF EXISTS Krankenhaus CASCADE;
DROP TABLE IF EXISTS Medikament CASCADE;"""
cur.execute(drop_tables)
conn.commit()
print("Alle Tabellen wurden gelöscht.")

Alle Tabellen wurden gelöscht.


In [ ]:
create_tables = """

-- =========================
-- CREATE TABLES
-- =========================

CREATE TABLE Krankenhaus (
    KH_ID INT PRIMARY KEY,
    Name VARCHAR(100) NOT NULL,
    Strasse VARCHAR(100) NOT NULL,
    Hausnummer VARCHAR(10) NOT NULL,
    PLZ VARCHAR(10) NOT NULL,
    Stadt VARCHAR(50) NOT NULL
);

CREATE TABLE Station (
    Station_ID INT PRIMARY KEY,
    Name VARCHAR(50) NOT NULL,
    Kapazitaet INT NOT NULL,
    KH_ID INT NOT NULL,
    FOREIGN KEY (KH_ID) REFERENCES Krankenhaus(KH_ID)
);

CREATE TABLE Patient (
    Patient_ID INT PRIMARY KEY,
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    Geburtsdatum DATE NOT NULL,
    Versicherungsnummer VARCHAR(30),
    Elektronisches_Patientendossier_ID VARCHAR(50),
    Strasse VARCHAR(100),
    Hausnummer VARCHAR(10),
    PLZ VARCHAR(10),
    Stadt VARCHAR(50),
    Station_ID INT,
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Arzt (
    Arzt_ID INT PRIMARY KEY,
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    Telefonnummer VARCHAR(30),
    ist_extern BOOLEAN NOT NULL,
    Station_ID INT,
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Pflegekraft (
    Pflegekraft_ID INT PRIMARY KEY,
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    Telefonnummer VARCHAR(30),
    Station_ID INT NOT NULL,
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Reserve (
    Reserve_ID INT PRIMARY KEY,
    Vorname VARCHAR(50),
    Nachname VARCHAR(50),
    Telefonnummer VARCHAR(30)
);

CREATE TABLE Bett (
    Bett_ID INT PRIMARY KEY,
    Status VARCHAR(20) NOT NULL,
    Station_ID INT NOT NULL,
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Behandlung (
    Behandlung_ID INT PRIMARY KEY,
    Patient_ID INT NOT NULL,
    Datum DATE NOT NULL,
    FOREIGN KEY (Patient_ID) REFERENCES Patient(Patient_ID)
);

CREATE TABLE Raum (
    Raum_ID INT PRIMARY KEY,
    Raum_Nr VARCHAR(20) NOT NULL,
    Art VARCHAR(50),
    Station_ID INT NOT NULL,
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Event (
    Event_ID INT PRIMARY KEY,
    Beschreibung VARCHAR(255),
    Datum DATE NOT NULL,
    Uhrzeit TIME NOT NULL,
    Raum_ID INT NOT NULL,
    Behandlung_ID INT NOT NULL,
    FOREIGN KEY (Raum_ID) REFERENCES Raum(Raum_ID),
    FOREIGN KEY (Behandlung_ID) REFERENCES Behandlung(Behandlung_ID)
    abgeschlossen BOOLEAN NOT NULL DEFAULT FALSE
);

CREATE TABLE Event_Arzt (
    Event_ID INT NOT NULL,
    Arzt_ID INT NOT NULL,
    PRIMARY KEY (Event_ID, Arzt_ID),
    FOREIGN KEY (Event_ID) REFERENCES Event(Event_ID),
    FOREIGN KEY (Arzt_ID) REFERENCES Arzt(Arzt_ID)
);

CREATE TABLE Medikament (
    Medikament_ID INT PRIMARY KEY,
    Name VARCHAR(100) NOT NULL,
    Wirkstoff VARCHAR(100)
);

CREATE TABLE Medikament_Event (
    Event_ID INT NOT NULL,
    Medikament_ID INT NOT NULL,
    Dosierung VARCHAR(50),
    Einheit VARCHAR(20),
    Zeitpunkt TIME,
    PRIMARY KEY (Event_ID, Medikament_ID),
    FOREIGN KEY (Event_ID) REFERENCES Event(Event_ID),
    FOREIGN KEY (Medikament_ID) REFERENCES Medikament(Medikament_ID)
);
"""

cur.execute(create_tables)
conn.commit()
print("Alle Tabellen wurden neu erstellt.")



Alle Tabellen wurden neu erstellt.


In [4]:
conn.rollback()
